In [ ]:
library(data.table)
library(dplyr)
library(stringr)
library(tidyr)
library(readxl)

###### Mutational Signature
ref_genome <- "BSgenome.Hsapiens.UCSC.hg38"

library(ref_genome, character.only = TRUE)

library("MutationalPatterns")
options(stringsAsFactors = F)

library(GenomicRanges)
library(Rsamtools)
library(MASS)
library(VariantAnnotation)

###
library(Biostrings)
library(BSgenome)

In [ ]:
setwd("/ch_progression/aric/passenger/")
## plot function
source("/tools/longitudinal-profiling-of-clonal-hematopoiesis/determinants_of_CH/environmental/hitchhiker/plot_mut_spectrum.R")

In [ ]:
# rm(list=ls())

load("synon_base.qcd_filtered.202406.rda")

ls()


In [ ]:
## plot VAF by CHIP status
longitudinalARIC.synon_base.qcd_filtered <- merge(synon_base.qcd_filtered, 
                                                  aric_baseline_n_v05.clones, 
                                                  by.x="gwasid", by.y="GWAS_ID")


nrow(longitudinalARIC.synon_base.qcd_filtered)

In [ ]:
### Baseline
ch_var_baseline <- fread("/ch_progression/aric/pheno/ch_var_in_baseline_qcd_pheno.23Mar2023.csv", header=T)

visitBase_nsSNV <- ch_var_baseline %>% 
  filter(GWAS_ID %in% aric_baseline_n_v05.clones$GWAS_ID & 
           ExonicFunc.refGene=="nonsynonymous SNV") %>% 
  summarise(nvar=n(), 
            n_samp=length(unique(GWAS_ID)) )

## unique Mutational spectrum for CHIP variants
plot_mut_spectrum(mutations = ch_var_baseline
                  [(ch_var_baseline$GWAS_ID %in% aric_baseline_n_v05.clones$GWAS_ID & 
                      ch_var_baseline$ExonicFunc.refGene=="nonsynonymous SNV"),c(5:8)], 
                  save = "FigS10a.NonsynonymousCHIPvBaseline.mut_spectrum.20240607.pdf",
                  genomeFile = ref_genome,
                  tissue = "Blood", 
                  add_mut_type = "Nonsynonymous CHIP ",
                  add_to_title = paste0(" (in ", visitBase_nsSNV$n_samp, " baseline visit paricipants)"),
                  unique_var = FALSE)


In [ ]:

  ## CHIP at follow-up visit
ch_var_05 <- fread("/ch_progression/aric/pheno/ch_var_in_v05_qcd_pheno.23Mar2023.csv", header=T)
## 
visit05_nsSNV <- ch_var_05 %>% 
  filter(GWAS_ID %in% aric_baseline_n_v05.clones$GWAS_ID &
           ExonicFunc.refGene=="nonsynonymous SNV") %>% 
  summarise(nvar=n(), 
            n_samp=length(unique(GWAS_ID)) )

## unique Mutational spectrum for CHIP variants
plot_mut_spectrum(mutations = ch_var_05
                  [(ch_var_05$GWAS_ID %in% aric_baseline_n_v05.clones$GWAS_ID & 
                      ch_var_05$ExonicFunc.refGene=="nonsynonymous SNV"),
                    c(5:8)], 
                  save = "FigS10b.NonsynonymousCHIPv05.mut_spectrum.20240607.pdf",
                  genomeFile = ref_genome,
                  tissue = "Blood", 
                  add_mut_type = "Nonsynonymous CHIP ",
                  add_to_title = paste0(" (in ", visit05_nsSNV$n_samp, " follow-up visit paricipants)"),
                  unique_var = FALSE)

In [ ]:

### plot 
# Passenger mut spectrum in chip +ve
N_CHIPpve_sSNV <- longitudinalARIC.synon_base.qcd_filtered %>% 
  filter(CHIP_baseline_or_visit05==1) %>% 
  summarise(nvar=n(), 
            n_samp=length(unique(gwasid)) 
            )
# plot mut pattern
plot_mut_spectrum(mutations = longitudinalARIC.synon_base.qcd_filtered
                  [longitudinalARIC.synon_base.qcd_filtered$CHIP_baseline_or_visit05==1, 
                                                                       c(3,4,6,7)], 
                  save = "FigS10c.passenger_mut_spectrum.CHIPcarrier.pdf",
                  genomeFile = ref_genome, 
                  tissue = "Blood", 
                  add_mut_type = "Synonymous Passenger (VAF: 1%-25%) ",
                  add_to_title = paste(" (in ", N_CHIPpve_sSNV$n_samp," paricipants with CHIP)"), 
                  unique_var =  FALSE)




In [ ]:
# Passenger mut spectrum in chip -ve
N_CHIPnve_sSNV <- longitudinalARIC.synon_base.qcd_filtered %>% 
  filter(CHIP_baseline_or_visit05==0) %>% 
  summarise(nvar=n(), 
            n_samp=length(unique(gwasid)) )

# plot mut pattern
plot_mut_spectrum(mutations = longitudinalARIC.synon_base.qcd_filtered
                  [longitudinalARIC.synon_base.qcd_filtered$CHIP_baseline_or_visit05==0, 
                                                                       c(3,4,6,7)], 
                  save = "FigS10d.passenger_mut_spectrum.nonCHIPcarrier.pdf",
                  genomeFile = ref_genome, 
                  tissue = "Blood", 
                  add_mut_type = "Synonymous Passenger (VAF: 1%-25%) ",
                  add_to_title = paste(" (in ", N_CHIPnve_sSNV$n_samp," paricipants without CHIP)"), 
                  unique_var =  FALSE)


